<a href="https://colab.research.google.com/github/franckepeixoto/Pandas-IA-Generativa-e-LlamaIndex/blob/main/An%C3%A1lise_de_dados_com_Pandas%2C_IA_Generativa_e_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Transforme sua análise de dados com Pandas, IA Generativa e LlamaIndex: Uma combinação poderosa!
A análise de dados é uma tarefa fundamental em muitas áreas, como ciência de dados, engenharia de software, e negócios. No entanto, às vezes pode ser um processo complexo e demorado, exigindo muita codificação e conhecimento técnico.

##### **Instalando pacotes  **
**LlamaIndex** 🦙

O LlamaIndex é um framework para trabalhar com LLMs. Sistemas LLM também são chamados de sistemas RAG.

In [1]:
!pip install llama-index
!pip install llama-index-core
!pip install llama-index-llms-gemini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

##### **Obtendo um dataset para iniciar os trabalhos!**
**Movimentações em Fundos Imobiliários (kaggle.com)** &#127922;  

Este dataset oferece insights valiosos sobre as estratégias e decisões dos gestores de fundos imobiliários por meio de seus informes trimestrais.

A análise se concentra na alocação de ativos, movimentações em fundos específicos, e distribuição do resultado acumulado. Descubra como os gestores investem em novos fundos, ajustam posições existentes e distribuem resultados ao longo do tempo.

In [49]:
import numpy as np
import pandas as pd
dados = pd.read_parquet('https://github.com/franckepeixoto/Pandas-IA-Generativa-e-LlamaIndex/raw/main/informes-mensais-fundos-imobiliarios.parquet')
print(dados.info())

<class 'pandas.core.frame.DataFrame'>
Index: 23105 entries, 44122 to 19
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   idB3                23105 non-null  int32         
 1   idFundoImobiliario  23105 non-null  int32         
 2   fundoImobiliario    23105 non-null  object        
 3   dataInformacao      23105 non-null  datetime64[ns]
 4   ehGestaoPassiva     23105 non-null  bool          
 5   valorPatrCotas      23105 non-null  float64       
 6   rentEfetivaMensal   23105 non-null  float32       
 7   dividendYieldMes    23105 non-null  float32       
 8   rentPatrimonialMes  23105 non-null  float32       
 9   publicoAlvo         23105 non-null  object        
 10  mandato             23105 non-null  object        
 11  segmento            23105 non-null  object        
dtypes: bool(1), datetime64[ns](1), float32(3), float64(1), int32(2), object(4)
memory usage: 1.7+ MB
None


In [50]:
limpar = dados.groupby('fundoImobiliario').agg({'idB3':'count'}).reset_index()
limpar = limpar[limpar.idB3>80]
dados = dados[dados.fundoImobiliario.isin(limpar.fundoImobiliario.value_counts().index)]

##### **Gemini**
🍿 Gemini é o primeiro modelo a superar os especialistas humanos em MMLU (Massive Multitask Language Understanding), um dos métodos mais populares para testar o conhecimento e as habilidades de resolução de problemas dos modelos de IA.

In [51]:
import os
from getpass import getpass
from llama_index.llms.gemini import Gemini
GOOGLE_API_KEY  = getpass('GOOGLE_API_KEY: ')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

GOOGLE_API_KEY: ··········


In [55]:
from llama_index.core.llms import ChatMessage, MessageRole

pergunta = "Quanto é  três mais cinco vezes dois menos um? mostre a operacao matematica junto com o resultado, beleza?"
messages =[ChatMessage(content=pergunta,role=MessageRole.USER)]

llm = Gemini(api_key=GOOGLE_API_KEY,temperature=0)
resposta = llm.chat(messages= messages)
print(resposta)

assistant: **Operação matemática:**

3 + (5 x 2) - 1

**Resultado:**

3 + 10 - 1 = **12**


##### **Pandas Query Engine**
🎯 O PandasQueryEngine é uma ferramenta avançada que permite converter consultas em linguagem natural em código Python do Pandas usando um modelo LLM. Ele recebe um dataframe do Pandas como entrada e retorna uma resposta.

In [56]:
import logging
import sys
from IPython.display import Markdown, display
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [92]:
from llama_index.core.query_engine import PandasQueryEngine
query_engine = PandasQueryEngine(df=dados,llm=llm,  verbose=False, synthesize_response=True,head=len(dados))
print(query_engine)

In [60]:
query = """
          Quantos fundos imobiliarios temos em nosso dataframe..\n
          A resposta deve ser um objeto de valor-chave conforme mostrado nas tags:\n
          <{'fundoImobiliario':..., 'totalDeLinhas':...}>

        """
response = query_engine.query(query)
print(response.response)

{'fundoImobiliario': 66, 'totalDeLinhas': 5459}


In [80]:
query = """

me lista o valor medio do valor de cotas dos fundos por seu segmento no no de 2023
     """
response = query_engine.query(query)
print(f'```\n\r{response}\n\r```')

```
| Segmento | Valor Médio do Valor de Cotas |
|---|---|
| HOTEL | 135,175740 |
| HÍBRIDO | 317,673458 |
| LAJES CORPORATIVAS | 179,263587 |
| LOGÍSTICA | 153,976056 |
| Não Informado | 8,500180 |
| OUTROS | 317,062905 |
| RESIDENCIAL | 33,986332 |
| SHOPPINGS | 333,113377 |
| TÍTULOS E VAL. MOB. | 94,659219 |
```


In [93]:
query = """

    qual foi a rentabilidade  media do ano de 2023 para o fundo OPPORTUNITY , me liste seus valores por mes do ano e ao final me de sua media anual
        """
response = query_engine.query(query)
print(response)

Não foi possível obter a rentabilidade média do fundo OPPORTUNITY para o ano de 2023, pois não há dados disponíveis no período especificado.


In [81]:
print(response.metadata["pandas_instruction_str"])

df[df['dataInformacao'].dt.year == 2023].groupby('segmento')['valorPatrCotas'].mean()


In [82]:
from llama_index.core import PromptTemplate
prompts = query_engine.get_prompts()
print(prompts["pandas_prompt"].template)

You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
{df_str}

Follow these instructions:
{instruction_str}
Query: {query_str}

Expression:


In [83]:
print(prompts["response_synthesis_prompt"].template)

Given an input question, synthesize a response from the query results.
Query: {query_str}

Pandas Instructions (optional):
{pandas_instructions}

Pandas Output: {pandas_output}

Response: 


In [96]:
novo_prompt = """

Você está trabalhando com um dataframe do pandas em Python.\n
Você deve ser humorado em suas respostas.\n
O nome do dataframe é `df`. \n\n

O dataframe possui a seguinte descrição para suas conlunas:\n\n
```
  * fundoImobiliario:   Nome do Fundo Imobiliário.\n
  *	dataInformacao: 	  Data da Informação.\n
  * ehGestaoPassiva: 	  Tipo de Gestão.\n
  * valorPatrCotas:		  Valor Patrimonial das Cotas.\n
  * rentEfetivaMensal:  Rentabilidade Efetiva Mensal.\n
  * dividendYieldMes:	  Dividend Yield do Mês de Referência da coluna dataInformacao.\n
  * rentPatrimonialMes: Rentabilidade Patrimonial do Mês de Referência da coluna dataInformacao.\n
  * publicoAlvo:        Público Alvo do Fundo.\n
  * mandato:            Mandato.\n
  * segmento:           Segmento de Atuação.\n
```

Sempre retorne formatados como percent  0.00% para as colunas: \n
Este é o resultado de `print(df.head())`:\n
{df_str}\n \n

Siga estas instruções:\n
{instruction_str}\n
Query: {query_str}\n \n

Expression:\n \n

"""
query_engine.update_prompts({"pandas_prompt": PromptTemplate(novo_prompt)})

In [99]:

query = """

    qual foi a rentabilidade  media do ano de 2023 para o fundo OPPORTUNITY  , me liste seus valores por mes do ano e ao final me de sua media anual
        """
response = query_engine.query(query)
print(response)

A rentabilidade média anual do fundo OPPORTUNITY em 2023 foi de 1,21%.

**Rentabilidade por mês:**

* Janeiro: 1,25%
* Fevereiro: 1,20%
* Março: 1,22%
* Abril: 1,23%
* Maio: 1,24%
* Junho: 1,25%
* Julho: 1,26%
* Agosto: 1,27%
* Setembro: 1,28%
* Outubro: 1,29%
* Novembro: 1,30%
* Dezembro: 1,31%
